# Battleships

In [18]:
# Library imports
import random
from getpass import getpass
import sys

import IPython
from IPython.display import display, clear_output

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout

In [29]:
# Functions (19)

def player_names():
    """Asks for the names of the players and returns both in a tuple."""
    player_1 = input("Player name: ")
    player_2 = input("Player name: ")
    return player_1, player_2

def choose_first_player(player_1_name, player_2_name):
    """Randomly selects which player has the first turn. Returns one of the two player names that were 
    passed to the function."""
    if random.randint(0,1) == 0:
        return player_1_name
    else:
        return player_2_name

def display_boards(player_name, radar_lst, sea_lst):
    """Prints the radar_lst and sea_lst lists in the format of Battleships boards, with the letters list
    denoting the rows."""
        
    print(player_name, "\n")
    print("Radar\n")
#     print("     1   2   3   4   5   6   7   8   9   10")
    print("     1  2   3  4   5  6  7   8   9  10")
    for index, i in enumerate(radar_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
    print("\nSea\n")
#     print("     1   2   3   4   5   6   7   8   9   10")
    print("     1  2   3  4   5  6  7   8   9  10")
    for index, i in enumerate(sea_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
def update_board_placement(sea_lst, boat_coords, symbol):
    """Updates the board after the placement of a boat, taking a list of boat_coords as input."""
    for coord in boat_coords:
        sea_lst[coord] = symbol
        
def update_board_combat(radar_lst, sea_lst, position, flag):
    """Taking the position and flag results from a turn, this function updates the radar_lst and 
    sea_lst lists that make up the board."""
    if flag:
        radar_lst[position] = radar_hit
        sea_lst[position] = sea_hit
    else:
        radar_lst[position] = radar_miss
        sea_lst[position] = sea_miss
        
def update_scoreboard():
    """Updates the scoreboard."""
    with scoreboard:
        clear_output()
        print("Scoreboard: \n")
        print(f"{player_1_name} has sunk:" + "\n"*(5-len(player_2_sunk_boats)))
        if player_2_sunk_boats:
            display(player_2_sunk_boats)
        print(f"{player_2_name} has sunk:")
        if player_1_sunk_boats:
            display(player_1_sunk_boats)
            
def enforce_player_coord(coord):
    """Ensures a player coordinate input is in the valid 'A1' - 'J10' format. Returns coord once in the correct format."""
    # Ensure second item in coord is a number
    if coord.lower() == "exit":
        sys.exit("Exited Battleships")
    
    try:
        int(coord[1])
    except (ValueError, IndexError):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()

    # Ensure coord is 2-3 characters long (and ends in '10') if 3 characters, starts with a letter in letters and
    # ends in a number 1-10
    while len(coord) > 3 or len(coord) < 2 or \
    (len(coord) == 3 and coord[1:3] != "10") or \
    coord[0] not in letters or int(coord[1]) not in range(1, 10):
        if coord.lower() == "exit":
            sys.exit("Exited Battleships")
        with text_update:
            print("Coordinates must be in format 'LetterNumber' (e.g. 'A1').")
        coord = input("Enter coordinates (in the format 'A1'): ").upper()
     
    return coord.upper()

def coord_string_to_digit(coord):
    """Takes a coordinate string as input and returns list index position."""
    return letters.index(coord[0]) * 10 + int(coord[1:])

def digit_to_coord_string(digit):
    """Takes a list index position as input and returns coordinate string."""
    number = digit % 10
    r = 0
    if number == 0:
        number = 10
        r = 1
    letter = letters[digit // 10 - r]
    return letter + str(number)

def place_boats(player_name, radar_lst, sea_lst, player_boat_coords):
    """Prompts user to place boats in descending order of length, by entering starting co-ordinate and direction.
    Calls check_space_free() to ensure boat placement is valid - if not, prompts user to place boat elsewhere.
    Saves placed boats in player_boats list."""
    
    for boat, data in boat_types.items():
        length, symbol = data
        coords = ""
        while not coords:            
            start_coord = coord_string_to_digit(enforce_player_coord(input(f"Enter start coordinate for {boat}: ").upper()))
            direction = ""
            first_dir = True
            while direction not in ["U","D","L","R"]:
                if not first_dir:
                    with text_update:
                        print("Direction must be one of 'U', 'D', 'L' or 'R'.")
                direction = input("Choose direction (U/D/L/R): ").upper()
                first_dir = False
            coords = check_space_free(sea_lst, start_coord, direction, length)
        player_boat_coords.append(coords)
        update_board_placement(sea_lst, coords, ship)

        with boards:
            clear_output()
            display(display_boards(player_name, radar_lst, sea_lst))
            
def check_space_free(sea_lst, start_coord, direction, length):
    """Checks if all required spaces for boat placement are free and in range.
    Returns tuple (coordinates, "Ship placed") if valid, otherwise returns (False, [error message])."""
    
    if sea_lst[start_coord] != sea_empty:
        with text_update:
            print("Sea already occupied here")
        return False
    
    boat_coords = [start_coord]
    row = start_coord // 10
    if start_coord % 10 == 0:
        row -= 1
                
    if direction in ["L", "R"]:
        d = 1
        if direction == "L":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * d)
            if (next_coord - 1) // 10 != row:
                with text_update:
                    print("Ship outside board")
                return False
            elif sea_lst[next_coord] != sea_empty:
                with text_update:
                    print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        with text_update:
            print("Ship placed")
        return boat_coords
    
    elif direction in ["U", "D"]:
        d = 1
        if direction == "U":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * 10 * d)
            if next_coord not in range(1, 101):
                with text_update:
                    print("Ship outside board")
                return False
            elif sea_lst[next_coord] != sea_empty:
                with text_update:
                    print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        with text_update:
            print("Ship placed")
        return boat_coords
    
    else:
        with text_update:
            print("Direction error")
        return False
    
def fire_shot(player_guesses):
    """Calls enforce_player_coord() to prompt player to enter a guess and enforce validity.
    If coordinates have been guessed by the player already, asks for another guess.
    If the guess is valid, adds the guess to player_guesses and returns the 
    list index ('position') of the player guess."""
    
    guess = ""
    guess = enforce_player_coord(guess)
    
    # Checks if guess has already been guessed in a previous turn
    while guess in player_guesses:
        with text_update:
            print("Coordinates already guessed!")
        guess = enforce_player_coord(input("Enter coordinates (in the format 'A1'): ").upper())
    
    player_guesses.append(guess) # adds guess to guess history
    position = coord_string_to_digit(guess)
    return position

def check_hit(sea_lst, position):
    """Takes player guess position and evaluates if it matched an opponent boat. Updates text_update and returns bool."""
#     if sea_lst[position] in sea_boats:
    if sea_lst[position] == ship:
        with text_update:
            print("Hit!")
        return True
    else:
        with text_update:
            print("Miss!")
        return False
    
def check_ship_status(sea_lst, player_boat_coords, player_sunk_boats):
    """Checks which ships are destroyed and updates ship count."""
    for index, boat in enumerate(player_boat_coords):
        if all([sea_lst[boat[i]] == sea_hit for i in range(len(boat))]):
            boat_type = list(boat_types.keys())[index]
            if boat_type not in player_sunk_boats:
                with text_update:
                    print(f"{boat_type} sunk!")
            player_sunk_boats.add(boat_type)
            
def winning_condition():
    """Updates global winner variable and returns True if either player has 5 ships sunk."""
    global winner
    if len(player_1_sunk_boats) == 5:
        winner = player_2_name
        return True
    elif len(player_2_sunk_boats) == 5:
        winner = player_1_name
        return True
    else:
        return False
    
def new_turn(name):
    """When turn ended (i.e. function is called), hides the previous player's board and asks next player if they are ready.
    Once second player is ready, sets turn to their go and displays their board."""
    
    global turn
    
    clear_output()
    display(app)

    boards.clear_output()
    with text_update:
        clear_output()
        print("Text updates:\n")
    ready = input(f"{name}, enter Y to continue: ").upper()
    while ready != "Y":
        if ready.lower() == "exit":
            sys.exit("Exited Battleships")
        ready = input(f"{name}, enter Y to continue: ").upper()
    turn = name

    clear_output()
    display(app)

    if name == player_1_name:
        with boards:
            display(display_boards(name, radar_lst_p1, sea_lst_p1))
    else:
        with boards:
            display(display_boards(name, radar_lst_p2, sea_lst_p2))
    
    with text_update:
        print(f"{name} - your go!")

def final_result(winner):
    """Displays congratulations message to winner and winner's board at the end of the game."""
    boards.clear_output()
    clear_output()
    display(app)
    
    if winner == player_1_name:
        with boards:
            display(display_boards(winner, radar_lst_p1, sea_lst_p1))
    else:
        with boards:
            display(display_boards(winner, radar_lst_p2, sea_lst_p2))
    
    print(f"\n\n{winner} wins, congratulations!\n")
    
def ask_replay():
    replay = ""
    while replay not in ["Y", "N"]:
        replay = input("Do you want to play again (Y/N)? ").upper()

    return replay

def test_variables():
    """Returns test variables for two players in the order:
    - test_radar
    - test_sea
    - test_boat_coords,
    - test_sunk_boats
    - test_guesses"""
    
    test_radar = [radar_empty]*101
    test_sea = [sea_empty] * 101
    test_radar2 = [radar_empty]*101
    test_sea2 = [sea_empty] * 101
    
    test_boat_coords = [[1,2,3,4,5], [11,12,13,14], [21,22,23], [31,32,33], [41,42]]
    test_boat_coords2 = [[100,90,80,70,60], [99,89,79,69], [98,88,78], [97,87,77], [96,86]]
    
    test_sunk_boats = set()
    test_guesses = []
    test_sunk_boats2 = set()
    test_guesses2 = []

    
    # Set player1_sea and player2_radar to all boats coords hit except last for each boat, and misses for opposite boards
    for boat, coords in enumerate(test_boat_coords):
        for i in range(len(coords)-1):
            test_sea[coords[i]] = sea_hit
            test_radar2[coords[i]] = radar_hit
            test_sea2[coords[i]] = sea_miss
            test_radar[coords[i]] = radar_miss
            test_guesses2.append(digit_to_coord_string(coords[i]))
#         test_sea[coords[len(coords)-1]] = sea_boats[boat]
        test_sea[coords[len(coords)-1]] = ship
    
    for boat, coords in enumerate(test_boat_coords2):
        for i in coords:
#             test_sea2[i] = sea_boats[boat]
            test_sea2[i] = ship
    
    
    return test_radar, test_sea, test_boat_coords, test_sunk_boats, test_guesses, \
            test_radar2, test_sea2, test_boat_coords2, test_sunk_boats2, test_guesses2

In [5]:
# def fire_input_return(player_guesses, change):
#     """Takes player_guesses and the change in the fire_input box as input, returns the position."""
#     player_guesses.append(change['new']) # adds guess to guess history
#     position = coord_string_to_digit(change['new'])
#     return position
# #     check_hit(position, sea_lst)

In [6]:
# def test_shot(sea_lst, player_guesses):
#     """Evaluates available coordinates, displays them in fire_input combobox."""
#     player_guess_positions = [coord_string_to_digit(guess) for guess in player_guesses]
    
#     avail_positions = [option for option, sea in enumerate(sea_lst) if option not in player_guess_positions]
#     avail_coords = [digit_to_coord_string(position) for position in avail_positions]
#     avail_coords = avail_coords[1:]
    
#     with user_inputs:
#         fire_input.options=avail_coords
#         display(fire_input)       

In [34]:
# Display containers
text_update = widgets.Output()
boards = widgets.Output()
scoreboard = widgets.Output()
# user_inputs = widgets.Output()

app =  AppLayout(header=None,
                 left_sidebar=text_update,
                 center=boards,
                 right_sidebar=scoreboard,
                 footer=None,
                )

print("Welcome to Battleships!\n")
# print("'x' is a hit, 'o' is a miss!")
display(app)

want_replay = "Y"
while want_replay == "Y":
    ### Game starts - setup ###
    
    with text_update:
        clear_output()
    with boards:
        clear_output()
    with scoreboard:
        clear_output()
    
    # Reset boards and define initial variables
    letters = ["A","B","C","D","E","F","G","H","I","J"]

    # Unicode characters
    green_square = "\U0001F7E9"
    wave = "\U0001F30A"
    ship = "\U0001F6A2"
    bomb = "\U0001F4A3"
    explosion = "\U0001F4A5"
    
    # Radar squares
#     radar_empty = "   "
    radar_empty = green_square
#     radar_boats = [" D ", " C ", " S ", " B ", " A "]
#     radar_hit = " x "
    radar_hit = explosion
#     radar_miss = " o "
    radar_miss = bomb

    # Sea squares
    
#     sea_empty = "~~~"
    sea_empty = wave
#     sea_boats = ["~A~", "~B~", "~S~", "~C~", "~D~"]
#     sea_hit = "~x~"
    sea_hit = explosion
#     sea_miss = "~o~"
    sea_miss = bomb

    # Boat types
    boat_types = {"Aircraft carrier (5)" : (5, "~A~"), 
                  "Battleship (4)" : (4, "~B~"), 
                  "Submarine (3)" : (3, "~S~"), 
                  "Cruiser (3)" : (3, "~C~"), 
                  "Destroyer (2)" : (2, "~D~")}

    # Create empty grids
    sea_lst_p1 = [sea_empty] * 101  
    radar_lst_p1 = [radar_empty] * 101
    sea_lst_p2 = [sea_empty] * 101  
    radar_lst_p2 = [radar_empty] * 101

    # Set of player boats that have been sunk
    player_1_sunk_boats = set()
    player_2_sunk_boats = set()

    # Lists of player boat locations
    player_1_boat_coords = []
    player_2_boat_coords = []

    # Lists of past guesses
    player_1_guesses = []
    player_2_guesses = []


    ### Play starts ###
    winner = False
    turn = ""
    
    # Ask for player names
    player_1_name, player_2_name = player_names()

    with text_update:
        print("Text updates:\n")
    with scoreboard:
        print("Scoreboard:\n")
        print(f"{player_1_name} has sunk:" + "\n"*5)
        print(f"{player_2_name} has sunk:")

    # Allow endgame testing
    if player_1_name.lower() == "test":
        radar_lst_p1, sea_lst_p1, player_1_boat_coords, player_1_sunk_boats, player_2_guesses, \
        radar_lst_p2, sea_lst_p2, player_2_boat_coords, player_2_sunk_boats, player_2_guesses = test_variables()
        turn == player_2_name
        with boards:
            display_boards(player_2_name, radar_lst_p2, sea_lst_p2)
        with text_update:
            print(f"{player_2_name} - your go!")

    else:
        # Determine which player starts
        turn = choose_first_player(player_1_name, player_2_name)    
        with text_update:
            print(f"\n{turn} starts!\n")

        # Players placing the boats
        if turn == player_1_name:
            # Player 1 places ships
            with boards:
                display_boards(player_1_name, radar_lst_p1, sea_lst_p1)
            place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_2_name}? ").upper()
            new_turn(player_2_name)
            # Player 2 places ships
            place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_1_name}? ").upper()
            new_turn(player_1_name)
        else:
            # Player 2 places ships
            with boards:
                display_boards(player_2_name, radar_lst_p2, sea_lst_p2)
            place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_1_name}? ").upper()
            new_turn(player_1_name)
            # Player 1 places ships
            place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_2_name}? ").upper()
            new_turn(player_2_name)

    # Taking turns to fire
    while not winning_condition():
        if turn == player_1_name:
        # Player 1 turn to fire
            # Get index position of player guess
            position = fire_shot(player_1_guesses)
    #         fire_input.observe(functools.partial(fire_input_return, player_1_guesses), 'value')
    #         position = fire_input_return()
            # Check if player guess was a hit
            flag = check_hit(sea_lst_p2, position)
            # Update Player 1 radar and Player 2 sea with result of shot
            update_board_combat(radar_lst_p1, sea_lst_p2, position, flag)
            # Check if any of Player 2's ships sunk, update sunk_ships list 
            check_ship_status(sea_lst_p2, player_2_boat_coords, player_2_sunk_boats)
            update_scoreboard()
            # Check if winning condition fulfilled
            if winning_condition():
                final_result(winner)
                break
            # Display updated Player 1 boards
            with boards:
                clear_output()
                display_boards(player_1_name, radar_lst_p1, sea_lst_p1)

            # Call new_turn() to initiate change of turn
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_2_name}? ").upper()
            new_turn(player_2_name)

        else:
        # Player 2 turn to fire
            # Get index position of player guess
            position = fire_shot(player_2_guesses)
    #         fire_input.observe(functools.partial(fire_input_return, player_2_guesses), 'value')
            # Check if player guess was a hit
            flag = check_hit(sea_lst_p1, position)
            # Update Player 2 radar and Player 1 sea with result of shot
            update_board_combat(radar_lst_p2, sea_lst_p1, position, flag)
            # Check if any of Player 1's ships sunk, update sunk_ships list 
            check_ship_status(sea_lst_p1, player_1_boat_coords, player_1_sunk_boats)
            update_scoreboard()
            # Check if winning condition fulfilled
            if winning_condition():
                final_result(winner)
                break
            # Display updated Player 2 boards
            with boards:
                clear_output()
                display_boards(player_2_name, radar_lst_p2, sea_lst_p2)

            # Call new_turn() to initiate change of turn
            ready = ""
            while ready != "Y":
                if ready.lower() == "exit":
                    sys.exit("Exited Battleships")
                ready = input(f" Ready to pass to {player_1_name}? ").upper()
            new_turn(player_1_name)

    
    
    want_replay = ask_replay()
    if want_replay == "N":
        break

AppLayout(children=(Output(layout=Layout(grid_area='left-sidebar')), Output(layout=Layout(grid_area='right-sid…



Kostis wins, congratulations!

Do you want to play again (Y/N)? n
